In [1]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>#notebook-container { margin-left:-14px; width:calc(100% + 27px) !important; }</style>"))

In [115]:
import csv, json, math, os, numbers, pandas, re, scipy, scipy.sparse, shutil
import struct, subprocess, sys, threading, time, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

In [118]:
def get_segments_from_ring(r):
    ret = set()
    for i in range(0, len(r)):
        ret.add((r[i - 1][0], r[i - 1][1], r[i][0], r[i][1]))
    return ret

def get_segments_from_polygon(p):
    return set.union(*[get_segments_from_ring(r) for r in p])

def get_segments_from_multipolygon(mp):
    return set.union(*[get_segments_from_polygon(p) for p in mp])

def get_segments(g):
    if 'features' in g:
        return set.union(*[get_segments(f) for f in g['features']])
    elif 'geometry' in g:
        if g['geometry']['type'] == 'Polygon':
            return get_segments_from_polygon(g['geometry']['coordinates'])
        elif g['geometry']['type'] == 'MultiPolygon':
            return get_segments_from_multipolygon(g['geometry']['coordinates'])
        else:
            raise Exception('unrecognized geometry type %s' % g['geometry']['type'])
    else:
        raise 'unrecognized type'
        
def LonLatToWebMercator(lon, lat):
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def WebMercatorToLonLat(x,y):
    lat = math.atan(math.exp((128.0 - y) * math.pi / 128.0)) * 360.0 / math.pi - 90.0
    lon = x * 360.0 / 256.0 - 180.0
    return [lon, lat]

def project_and_binarize_segment(segment, dest):
    (x1, y1) = LonLatToWebMercator(segment[0], segment[1])
    (x2, y2) = LonLatToWebMercator(segment[2], segment[3])
    dest.write(struct.pack('<ffff', x1, y1, x2, y2))

def binarize_geojson_outlines(src, dest):
    gj = json.load(open(src))
    print 'Read %d features from %s' % (len(gj['features']), src)
    segments = get_segments(gj)
    print '%d segments' % len(segments)

    try:
        os.makedirs(os.path.dirname(dest))
    except OSError:
        pass
        
    out = open(dest + '.tmp', 'w')
    for segment in segments:
        project_and_binarize_segment(segment, out)
    out.close()
    os.rename(dest + '.tmp', dest)
    print 'Created %s (%d segments)' % (dest, os.stat(dest).st_size / 16)
        

In [125]:
#!wget https://www2.census.gov/geo/tiger/TIGER2010/TRACT/2010/tl_2010_42003_tract10.zip
#!/usr/bin/ogr2ogr -f GeoJSON -t_srs crs:84 allegheny_county/tl_2010_42003_tract10.geojson allegheny_county/tl_2010_42003_tract10.shp

binarize_geojson_outlines('allegheny_county/tl_2010_42003_tract10.geojson',
                          'allegheny_county/allegeny_county_tracts_2010.bin')

Read 402 features from allegheny_county/tl_2010_42003_tract10.geojson
118186 segments
Created allegheny_county/allegeny_county_tracts_2010.bin (118186 segments)


In [126]:
#!/usr/bin/ogr2ogr -f GeoJSON -t_srs crs:84 allegheny_county/tl_2010_42003_bg10.geojson allegheny_county/tl_2010_42003_bg10.shp

binarize_geojson_outlines('allegheny_county/tl_2010_42003_bg10.geojson',
                          'allegheny_county/allegeny_county_blockgroups_2010.bin')

Read 1100 features from allegheny_county/tl_2010_42003_bg10.geojson
220899 segments
Created allegheny_county/allegeny_county_blockgroups_2010.bin (220899 segments)


In [127]:
#!/usr/bin/ogr2ogr -f GeoJSON -t_srs crs:84 allegheny_county/tl_2010_42003_tabblock10.geojson allegheny_county/tl_2010_42003_tabblock10.shp

binarize_geojson_outlines('allegheny_county/tl_2010_42003_tabblock10.geojson',
                          'allegheny_county/allegeny_county_blocks_2010.bin')

Read 30519 features from allegheny_county/tl_2010_42003_tabblock10.geojson
1141888 segments
Created allegheny_county/allegeny_county_blocks_2010.bin (1141888 segments)


In [128]:
def numpy_memmap_read(path, dtype):
    nelems = os.stat(path).st_size / numpy.dtype(dtype).itemsize
    return numpy.memmap(path, dtype=dtype, shape=(nelems,), mode='r')

numpy_memmap_read('allegheny_county/allegeny_county_tracts_2010.bin', numpy.float32)

memmap([ 71.16960907,  96.48995209,  71.1697464 , ...,  96.5345459 ,
        71.14060211,  96.5345993 ], dtype=float32)